### Building a RAG pipeline

In this notebook, we will learn

- <b>Setting up the environment</b>:
    - Configure environment variables for LangChain, OpenAI, and Pinecone
    - Understand the importance of API keys and project settings

- <b>Importing necessary libraries</b>:
    - Learn about key imports from langchain, including core components and specific modules

- <b>Configuring the retrieval setup</b>:
    - Set constants for index name, number of top results, and document metadata

- <b>Initializing key components</b>:
    - Create OpenAI embeddings
    - Set up the language model (ChatOpenAI)
    - Initialize the vector store (PineconeVectorStore)

- <b>Creating a retriever</b>:
    - Convert the vector store into a retriever with specific search parameters

- <b>Designing the chat template</b>:
    - Understand the structure of ChatPromptTemplate
    - Learn how to create system and human messages for the chatbot

- <b>Implementing helper functions</b>:
    - Create a function to format retrieved documents

- <b>Building the RAG (Retrieval-Augmented Generation) chain</b>:
    - Understand the concept of RunnablePassthrough and RunnableParallel
    - Create a chain for processing formatted documents
    - Build a chain that combines retrieval and answer generation

- <b>Using LangChain Hub</b>:
    - Learn how to pull prompts from LangChain Hub
    - Understand how to integrate custom prompts into the RAG chain

- <b>Pushing prompts to LangChain Hub</b>:
    - Learn the process of pushing custom prompts to the hub for reuse

- <b>Practical application</b>:
    - Run example queries through the RAG chain
    - Analyze the output and understand how context retrieval enhances responses

- <b>Best practices and considerations</b>:
    - Discuss the importance of filtering and metadata in retrieval
    - Explore ways to optimize retrieval and response generation

- <b>Troubleshooting and debugging</b>:
    - Identify common issues in the retrieval pipeline
    - Learn techniques for debugging and improving performance

![Alt text](../images/retriever.png)

### Setting up keys

In [ ]:
# Import the 'os' module for interacting with the operating system
import os

# Set the LANGCHAIN_TRACING_V2 environment variable to enable tracing for LangChain version 2
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Set the LANGCHAIN_API_KEY environment variable with your LangChain API key
os.environ["LANGCHAIN_API_KEY"] = "<YOUR API KEY HERE>"

# Set the OPENAI_API_KEY environment variable with your OpenAI API key
os.environ["OPENAI_API_KEY"] = "<YOUR API KEY HERE>"

# Set the PINECONE_API_KEY environment variable with your Pinecone API key
os.environ["PINECONE_API_KEY"] = "<YOUR API KEY HERE>"

# Set the LANGCHAIN_PROJECT environment variable to specify the project name for LangChain
os.environ['LANGCHAIN_PROJECT'] = "<YOUR PROJECT NAME HERE>"

# Set the LANGSMITH_USER_HANDLE environment variable with your Langsmith user handle
os.environ['LANGSMITH_USER_HANDLE'] = "<YOUR USER HANDLE HERE>"

### Importing the required packages

In [ ]:
# Import the json module for handling JSON data
import json

# Import ChatOpenAI from langchain_openai for interacting with the OpenAI chat model
from langchain_openai import ChatOpenAI

# Import OpenAIEmbeddings from langchain_openai for working with OpenAI embeddings
from langchain_openai import OpenAIEmbeddings 

# Import RunnableParallel from langchain_core.runnables for running tasks in parallel
from langchain_core.runnables import RunnableParallel

# Import ChatPromptTemplate from langchain_core.prompts for creating chat prompt templates
from langchain_core.prompts import ChatPromptTemplate

# Import RunnablePassthrough from langchain_core.runnables for running tasks in a passthrough manner
from langchain_core.runnables import RunnablePassthrough

# Import StrOutputParser from langchain_core.output_parsers for parsing string outputs
from langchain_core.output_parsers import StrOutputParser

# Import PineconeVectorStore from langchain_pinecone for managing vector storage in Pinecone
from langchain_pinecone import PineconeVectorStore

### Defining global variables

In [ ]:
# Define the index name for storing or retrieving data, in this case, 'earning-calls'
INDEX_NAME = '<YOUR CODE HERE>'

# Set the number of top results to return, here set to 6
TOP_K = 6

# Specify the quarter for which the data is relevant, in this example, Q1 (Quarter 1)
QUARTER = "Q1"

# Define the filename of the document being processed, in this case, "Adani Enterprises Ltd.pdf"
FILENAME = "Adani Enterprises Ltd.pdf"

# Specify the fiscal year for the data, in this example, FY24 (Fiscal Year 2024)
YEAR = "FY24"

# Initialize OpenAIEmbeddings with the specified model for generating text embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Initialize ChatOpenAI with the specified model and temperature for generating chat responses
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

## Loading Vectorstore

In [ ]:
# Create a PineconeVectorStore instance with the specified index name and embedding model
index = PineconeVectorStore(index_name=INDEX_NAME, 
                            embedding=embeddings) 

# Convert the PineconeVectorStore instance into a retriever object for searching
retriver = <YOUR CODE HERE>

In [ ]:
retriver.invoke("what is the capex?")

## Creating a prompt template

In [ ]:
# This template is designed for a chatbot that acts as an expert Q&A system
chat_template = ChatPromptTemplate.from_messages(
    [
        # Define the system message that outlines the chatbot's role and rules
        (
        "system", """You are an expert Q&A system that is trusted around the world. Always answer the query using the 
        provided context information, and not prior knowledge. Some rules to follow:
        
        1. Never directly reference the given context in your answer.
        2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."""
        ),
        
        # Define the human message that provides context information and a query
        (
        "human", """Context information is below.
        \n---------------------\n
        {context}
        \n---------------------\n
        
        Given the context information and not prior knowledge, answer the query.
        Query: {query}
        Answer: """
        ),
    ]
)

In [ ]:
print(chat_template.format(context="This is a sample context to see how the prompt looks like", 
                           query="This is a sample query?"))

## Creating the RAG chain

In [ ]:
# Define a function to format documents by joining their page content with newlines
def format_docs(docs):
    # Join the page content of each document with two newlines between them
    return "\n\n".join(doc.page_content for doc in docs)

# Create a runnable chain for generating responses from formatted documents
# This chain starts with formatting the documents, then uses the chat template,
# processes the response through the language model, and finally parses the output
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | chat_template
    | llm
    | StrOutputParser()
)

# Create a runnable chain for retrieving context and generating answers in parallel
# This chain retrieves context using the retriever and formats the query,
# then generates answers using the previously defined chain for formatted documents
rag_chain_with_source = RunnableParallel(
    {"context": retriver, "query": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [ ]:
response = rag_chain_with_source.invoke("What was the income?")
print(response['answer'])

### Prompt Versioning

Loading a Specific Version of a Prompt:

1. **Version Tracking in Repositories:**
   - Each push to a prompt repository saves a new version, identified by a unique commit hash.

2. **Loading the Latest Version:**
   - By default, accessing the repo will load the most recent version of a given prompt.

3. **Loading a Specific Version:**
   - To load a specific version, include its commit hash with the prompt name.
   - Example: For loading the "earnings-call-rag" with version `6214c98a`, append this hash to the prompt name in your loading command.

In [ ]:
# !pip install langchainhub

In [ ]:
from langchain import hub
prompt = hub.pull("bhaskarjit/earnings-call-rag")

In [ ]:
print(prompt.format(context="This is a sample context to see how the prompt looks like", 
                    query="This is a sample query?"))

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain_from_docs = <YOUR CODE HERE>

rag_chain_with_source = RunnableParallel(
    {"context": retriver, "query": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [ ]:
response = rag_chain_with_source.invoke("What was the income?")
print(response['answer'])

### How to Share Prompts on LangChain Hub:

1. **Getting Started:**
   - Getting prompts from LangChain Hub is easy, and so is sharing your own prompts.

   - This lets you easily share and manage your own prompts.

2. **Making a Prompt:**

   - First, create a prompt that fits what you need.

   - Make sure it follows the rules of LangChain Hub.

3. **Sharing Process:**

   - The sharing has two important parts:
     - **Account Handle:** Your special name in LangChain Hub, like `me-langchain-user`.
     - **Prompt Name:** A clear name for your prompt, showing what it does.

4. **How to Share with Code:**
   - This is a simple way to share:
     ```python
     from langchain import hub

     # Define your prompt
     my_prompt = "..."  # Your prompt content goes here

     # Share it on LangChain Hub
     hub.push(f"{account_handle}/{prompt_name}", my_prompt)
     ```
     - Replace `account_handle` with your username and `my_prompt` with your prompt's name.
     
     - Make sure `my_prompt` follows LangChain PromptTemplate.

5. **Using Your Shared Prompt:**
   - Once it's shared, you can use it in different apps through LangChain Hub.
   - This makes it easy to share with others, work together, and manage your prompts.

In [ ]:
len(prompt.messages), prompt.messages

In [ ]:
prompt.messages[1]

In [ ]:
prompt.messages[1].prompt

In [ ]:
print(prompt.messages[1].prompt.template)

In [ ]:
prompt.messages[1].prompt.template = """Context information is below.
\n---------------------\n
{context}
\n---------------------\n
Given the context information and not prior knowledge, answer the query.
Use Bullet poits whenever possible in the answer.
Query: {query}
Answer: """

In [ ]:
print(prompt.messages[1].prompt.template)

Added the following line in the prompt: `Use Bullet poits whenever possible in the answer.`

In [ ]:
handle = "bhaskarjit"
prompt_url = hub.push(f"{handle}/av-earnings-call-rag", prompt)

In [ ]:
prompt_url